# Financial Econometrics: Homework 1
#### Kryštof Veřtát, Tomáš Svoboda

## Problem 1

### Part 1

In [ ]:
library(fGarch)
library(repr)
library(forecast)
library(aTSA)
library(rugarch)
library(tseries)
library(fDMA)
library(quantmod)

In [ ]:
symbols <- read.csv("symbols2.csv")

head(symbols)

set.seed(62635431)

rows <- sample(x = 1:377, size = 100, replace = F)

symbols_download <- as.character(symbols[rows,2])

In [ ]:
# Download data for selected period
data <- lapply(symbols_download, function(y) {
  tryCatch({
    getSymbols(y, auto.assign = FALSE, src = "yahoo", from = "2020-01-01", to = "2022-10-01")
  }, error = function(e) {
    message(paste("Error downloading data for", y, "-", e$message))
    return(NULL)
  })
})

In [ ]:
# Keep only closing prices and remove stocks that are not in quantmod package
data <- lapply(data, function(y){
  y <- y[,4]
})

data <- data[sapply(data, function(x) !is.null(x))]

# Calculate returns from the closing prices
returns <- lapply(data, function(y){
  na.omit(diff(log(y)))
})

### Part 2

In [ ]:
# Specify GARCH(1,1) model and estimate it for each stock
spec <- ugarchspec(mean.model = list(armaOrder=c(0, 0)), variance.model = list(garchOrder = c(1, 1)))
models <- lapply(returns, function(y){
                 ugarchfit(spec, y)})


### Part 3

In [ ]:
# Get alpha, beta and alpha+beta coefficients from the results of the model
alpha_values <- lapply(models, function(model) coef(model)[["alpha1"]])

beta_values <- lapply(models, function(model) coef(model)[["beta1"]])

alpha_beta <- unlist(alpha_values) + unlist(beta_values)

In [ ]:
# Plot histograms with the coefficients
hist(as.numeric(alpha_values), col = "orange")

hist(as.numeric(beta_values), col = "yellow")

hist(as.numeric(alpha_beta), col = "green")

Histograms show that alpha coefficients have lower values, mostly below 0.3, while beta has relatively high values above 0.6. Sum of alpha and beta is very close to 1, but slightly lower, so it fulfills necessary condition that $\alpha + \beta < 1$. Value of alpha being lower than value of beta indicates that shocks to volatility caused by increased conditionall variance are more persistant compared to those caused by returns.

### Part 4

In [ ]:
# Find minimum and maximum values for each coefficient
min(unlist(alpha_values))
max(unlist(alpha_values))

min(unlist(beta_values))
max(unlist(beta_values))

min(unlist(alpha_beta))
max(unlist(alpha_beta))

### Part 5

In [ ]:
# Get median and quantiles for each day
medians <- vector()
quantiles_95 <- vector()
quantiles_05 <- vector()

for (i in 1:692){
  returns_daily <- vector()
  returns_daily <- lapply(models, function(model)model@fit$sigma[i])
  medians[i] <- median(unlist(returns_daily))
  quantiles_05[i] <- quantile(unlist(returns_daily),probs = 0.05)
  quantiles_95[i] <- quantile(unlist(returns_daily),probs = 0.95)
}

In [ ]:
# Plot time series of median, 0.05 quantile and 0.95 quantile
plot.ts(medians, col = "blue")
plot.ts(quantiles_05, col = "green")
plot.ts(quantiles_95, col = "red")

Median, 95 % and 5 % quantile of the returns have approximately the same figure with different magnituted. We can see that all three have the highest volatility during the first 100 days, where it seems to be the most persitant. Then it lowering and there are only smaller spikes in volatility. Especially for 95 % quantile the spikes are relatively higher. 